# LRPD DEMO

## Content of Implementation 

### 1. Automated LRPD Shadow Matrices Marking (parse.py)

### 2. Automated LRPD Analysis (lrpd_test.py)

### 3. Automated Parallization Marked Doall Loop 

    3.1 Auto Parallization using OpenMP (convert_parallel.py)

    3.2 Auto Parallization using Pthreads (convert_pthreads.py)
### 4. BenchMarking Result

In [2]:
import os
# project_path = '/Users/pbhoopala/Documents/school/eecs/eecs583/EECS583_Project'
project_path = './'
os.getcwd()
os.chdir(project_path)

## 0. Example code

In [3]:
!ls ./test_example

example_reduction.c                simple_example.c
example_reduction_parallel.c       simple_example2.c
marked_simple_500_iter_pthreads.c  simple_example2_parallel.c
nested_for_loop_example.c          simple_example_parallel.c
nested_for_loop_example_parallel.c


In [4]:
example_name = "simple_example"
file_name = example_name +".c"

In [5]:
!cat ./test_example/{file_name}

int main() {
    int A[4] = {1, 2, 3, 4};
    for(int i=0; i<4; ++i){
        A[i] = i+1;
    }
}

## 1. Automated Shadow Matrices Marking 

### 1.1 Code intro

In [6]:
!cat parse.py

from pycparser import c_ast, parse_file, c_generator
import copy
import sys

global_array_size = 0
filename = sys.argv[1]

class FuncDefVisitor(c_ast.NodeVisitor):
    def __init__(self):
        self.array_decls = []

    def visit_Decl(self, node):
        global global_array_size
        if isinstance(node.type, c_ast.ArrayDecl) and node.name == 'A':
            shadow_names = ['Aw', 'Ar', 'Anp', 'Anx']
            array_size = node.type.dim.value
            global_array_size = node.type.dim.value

            for name in shadow_names:
                array_type = c_ast.ArrayDecl(type=c_ast.TypeDecl(declname=name, quals=[], align=None, type=c_ast.IdentifierType(names=['int'])), dim=c_ast.Constant(type='int', value=array_size), dim_quals=[])
                init_list = c_ast.InitList([c_ast.Constant(type='int', value='0') for _ in range(int(array_size))])

                array_decl = c_ast.Decl(name=name, quals=[], storage=[], funcspec=[], type=array_type, bitsize=None, init=init_l

### 1.2 Code run

In [13]:
!cat simple.c

int main() {
    int A[4] = {1, 2, 3, 4};
    
    int i;

    for (i = 0; i < 4; i++) {
        A[i] = A[i] + 5;
    }

    return 0;
}

In [11]:
!python3 parse.py simple.c

### 1.3 Show the Marked Code

In [12]:
!cat ./marked_examples/marked_simple.c

int main()
{
  int A[4] = {1, 2, 3, 4};
  int Aw[4] = {0, 0, 0, 0};
  int Ar[4] = {0, 0, 0, 0};
  int Anp[4] = {0, 0, 0, 0};
  int Anx[4] = {0, 0, 0, 0};
  int write_counter = 0;
  int distinct_write_counter = 0;
  int i;
  for (i = 0; i < 4; i++)
  {
    int Awi[4] = {0, 0, 0, 0};
    A[i] = A[i] + 5;
    if (Awi[i] == 0)
    {
      Ar[i] = 1;
      Anp[i] = 1;
    }
    Aw[i] = 1;
    Awi[i] = 1;
    Ar[i] = 0;
    write_counter += 1;
  }

  
    for (int i = 0; i < 4; ++i){
        if (Aw[i] == 1) ++distinct_write_counter;
    }

    for (int i = 0; i < 4; i++) {
        printf("%d ", Aw[i]);
    }
    printf("\n");

    for (int i = 0; i < 4; i++) {
        printf("%d ", Ar[i]);
    }
    printf("\n");

    for (int i = 0; i < 4; i++) {
        printf("%d ", Anx[i]);
    }
    printf("\n");

    for (int i = 0; i < 4; i++) {
        printf("%d ", Anp[i]);
    }
    printf("\n");

    printf("%d ", write_counter);
    printf("\n");
    printf("%d ", distinct_write_counter);

    re

## 2. LRPD Analysis

### 2.1 Code intro

In [14]:
!cat lrpd_test.py

import sys

file_path = sys.argv[1]
with open(file_path, 'r') as file:
    Aw = list(map(int, file.readline().split()))
    Ar = list(map(int, file.readline().split()))
    Anx = list(map(int, file.readline().split()))
    Anp = list(map(int, file.readline().split()))

    write_counter = int(file.readline())
    distinct_write_counter = int(file.readline())

def lrpd_test():
    for i in range(len(Aw)):
        if Ar[i] == Aw[i]: return True
    
    if write_counter == distinct_write_counter: return True

    for i in range(len(Aw)):
        if Aw[i] == Anp[i] == Anx[i]: return False
    
    return True

print(1) if lrpd_test() else print(0)

### 2.2 Code Run

In [27]:
!python3 lrpd_test.py

Traceback (most recent call last):
  File "/Users/pbhoopala/Documents/school/eecs/eecs583/EECS583_Project/lrpd_test.py", line 3, in <module>
    file_path = sys.argv[1]
IndexError: list index out of range


### 2.3 LRPD test results

## 3. Automated Parallization of Marked Doall Loop Using OpenMP

### 3.1 Code intro

In [16]:
!cat ./convert_parallel.py

import clang.cindex
import sys
clang.cindex.Config.set_library_path("/Library/Developer/CommandLineTools/usr/lib")

def get_print_lines(filename):
    print_line = []
    idx = clang.cindex.Index.create()
    tu = idx.parse(filename)
    for token in tu.cursor.get_tokens():
        #print(token.spelling)
        if(token.spelling == "print" or token.spelling == "printf"):
            #print(token.location.line)
            print_line.append(int(token.location.line) - 1)
    return print_line


def convert_serial_to_parallel(filename, print_line):
    output_file = open(filename.split(".")[0]+"_parallel.c", "w+")
    input_file = open(filename, "r")
    current_line_num = 0
    braces_num = 0
    start = False
    print_num = 0
    output_file.write("#include <omp.h>\n")
    for line in input_file:
        current_line_num += 1
        number_space = len(line) - len(line.lstrip())
        add_critical = False
        if(line.strip()[:3] == "for" and braces_num == 0):
            #print(

### 3.2 Code run

In [18]:
file_path = "./marked_examples/marked_simple.c"
! python3 ./convert_parallel.py {file_path}

^C


### 3.3 Parallized Code

In [19]:
!cat ./marked_examples/marked_simple_parallel.c

#include <omp.h>
#include <stdio.h>

int main()
{
  int A[4] = {1, 2, 3, 4};
  int Aw[4] = {0, 0, 0, 0};
  int Ar[4] = {0, 0, 0, 0};
  int Anp[4] = {0, 0, 0, 0};
  int Anx[4] = {0, 0, 0, 0};
  int write_counter = 0;
  int distinct_write_counter = 0;
  int i;
  #pragma omp parallel for
  for (i = 0; i < 4; i++){
    int Awi[4] = {0, 0, 0, 0};
    A[i] = A[i] + 5;
    if (Awi[i] == 0)
    {
      Ar[i] = 1;
      Anp[i] = 1;
    }
    Aw[i] = 1;
    Awi[i] = 1;
    Ar[i] = 0;
    #pragma omp critical
    {
    write_counter += 1;
    }
  }
  #pragma omp barrier

  
    #pragma omp parallel for
    for (int i = 0; i < 4; ++i){
        if (Aw[i] == 1)
        ++distinct_write_counter;
    }
    #pragma omp barrier

    for (int i = 0; i < 4; i++) {
        printf("%d ", Aw[i]);
    }
    printf("\n");

    for (int i = 0; i < 4; i++) {
        printf("%d ", Ar[i]);
    }
    printf("\n");

    for (int i = 0; i < 4; i++) {
        printf("%d ", Anx[i]);
    }
    printf("\n");

    for (in

### 3.4 Parallel code run

In [31]:
!gcc-13 -fopenmp -o ./marked_examples/marked_simple_parallel ./marked_examples/marked_simple_parallel.c

!./marked_examples/marked_simple_parallel > marked_examples/marked_simple_parallel.txt

!cat marked_examples/marked_simple_parallel.txt

1 1 1 1 
0 0 0 0 
0 0 0 0 
1 1 1 1 
4 
4 

## 4. Automated Parallization of Marked Doall Loop Using Pthreads

### 4.1 Code intro

In [22]:
!cat ./convert_pthreads.py

import sys
import re
import os

def convert_serial_to_pthreads(file_path, num_threads):
  lines = []

  with open(file_path, "r") as file:
    for line in file:
      lines.append(line.strip())
  
  new_lines = []
  new_lines.append("#define _POSIX_C_SOURCE 200112L")
  new_lines.append("#include <pthread.h>")
  new_lines.append("#include <stdio.h>")
  new_lines.append("#include <stdlib.h>")
  new_lines.append('#include "pthreads_barrier.h"')
  
  new_lines.append("#define ADD_LOCK_A(idx, code) pthread_mutex_lock(locks_A + (idx)); code pthread_mutex_unlock(locks_A + (idx));")
  new_lines.append("#define ADD_LOCK_Ar(idx, code) pthread_mutex_lock(locks_Ar + (idx)); code pthread_mutex_unlock(locks_Ar + (idx));")
  new_lines.append("#define ADD_LOCK_Aw(idx, code)  pthread_mutex_lock(locks_Aw + (idx)); code  pthread_mutex_unlock(locks_Aw + (idx));")
  new_lines.append("#define ADD_LOCK_Anp(idx, code) pthread_mutex_lock(locks_Anp + (idx)); code  pthread_mutex_unlock(locks_Anp + (idx));")
  ne

### 4.2 Code run

In [3]:
!python3 ./src/convert_parallel_pthreads.py ./marked_examples/marked_simple.c 2

python3: can't open file '/Users/zyuhao/Desktop/CS 583/final_proj/Proj_new/EECS583_Project/./src/convert_pthreads.py': [Errno 2] No such file or directory


### 4.3 Parallelized code

In [25]:
!cat ./marked_examples/marked_simple_pthreads.c

#define _POSIX_C_SOURCE 200112L
#include <pthread.h>
#include <stdio.h>
#include <stdlib.h>
#include "pthreads_barrier.h"
#define ADD_LOCK_A(idx, code) pthread_mutex_lock(locks_A + (idx)); code pthread_mutex_unlock(locks_A + (idx));
#define ADD_LOCK_Ar(idx, code) pthread_mutex_lock(locks_Ar + (idx)); code pthread_mutex_unlock(locks_Ar + (idx));
#define ADD_LOCK_Aw(idx, code)  pthread_mutex_lock(locks_Aw + (idx)); code  pthread_mutex_unlock(locks_Aw + (idx));
#define ADD_LOCK_Anp(idx, code) pthread_mutex_lock(locks_Anp + (idx)); code  pthread_mutex_unlock(locks_Anp + (idx));
#define ADD_LOCK_Anx(idx, code) pthread_mutex_lock(locks_Anx + (idx)); code   pthread_mutex_unlock(locks_Anx + (idx));
#define ADD_LOCK_dist_w(code) pthread_mutex_lock(&lock_dist_w); code   pthread_mutex_unlock(&lock_dist_w);
#define ADD_LOCK_w(code) pthread_mutex_lock(&lock_w); code   pthread_mutex_unlock(&lock_w);
#define NUM_ELEMS 4
#define NUM_ITER 4
#define NUM_THREADS 2
int A[4] = {1, 2, 3, 4};
int Aw[4] = {0,

### 4.4 Parallel code run

In [39]:
!gcc -o ./marked_examples/marked_simple_pthread ./marked_examples/marked_simple_pthreads.c -lpthread
!./marked_examples/marked_simple_pthread > marked_examples/marked_simple_pthread.txt
!cat marked_examples/marked_simple_pthread.txt

6 7 8 9 
1 1 1 1 
0 0 0 0 
0 0 0 0 
1 1 1 1 
4 
4 

## 5. Analysis of LRPD Test using `lrpd_test.py`

In [44]:
!cat lrpd_test.py

import sys

file_path, p_thread_indicator = sys.argv[1], sys.argv[2]
with open(file_path, 'r') as file:
    if p_thread_indicator == '1': file.readline()
    Aw = list(map(int, file.readline().split()))
    Ar = list(map(int, file.readline().split()))
    Anx = list(map(int, file.readline().split()))
    Anp = list(map(int, file.readline().split()))

    write_counter = int(file.readline())
    distinct_write_counter = int(file.readline())

def lrpd_test():
    for i in range(len(Aw)):
        if Ar[i] == Aw[i]: return True
    
    if write_counter == distinct_write_counter: return True

    for i in range(len(Aw)):
        if Aw[i] == Anp[i] == Anx[i]: return False
    
    return True

print('LRPD Test passed, loop is DOALL') if lrpd_test() else print('LRPD test failed, loop is NOT DOALL')

In [43]:
!python3 lrpd_test.py marked_examples/marked_simple_pthread.txt 1

LRPD Test passed, loop is DOALL
